In [1]:
#Check GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()) 

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1039182954703553623
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 18536267776
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12707614567726930981
physical_device_desc: "device: 0, name: NVIDIA RTX A4500, pci bus id: 0000:65:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [2]:
import re

from happytransformer import HappyGeneration
from happytransformer import GENSettings

happy_gen = HappyGeneration("GPT-NEO", "EleutherAI/gpt-neo-2.7B")

05/20/2022 16:19:23 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [3]:
def selectFromDict(options, name):

    index = 0
    indexValidList = []
    print('Select a ' + name + ':')
    for optionName in options:
        index = index + 1
        indexValidList.extend([options[optionName]])
        print(str(index) + ') ' + optionName)
    inputValid = False
    while not inputValid:
        inputRaw = input(name + ': ')
        inputNo = int(inputRaw) - 1
        if inputNo > -1 and inputNo < len(indexValidList):
            selected = indexValidList[inputNo]
            print('Selected ' +  name + ': ' + selected)
            inputValid = True
            break
        else:
            print('Please select a valid ' + name + ' number')

    return selected

In [4]:
def GenText(input_txt):
    tmp = [0.65, 0.75, 0.80, 0.85, 0.90, 0.95]
    resp = []
    for temp in tmp:
        top_k_sampling_settings = GENSettings(do_sample=True, 
                                                top_k=200, 
                                                temperature=temp,  
                                                max_length=100, 
                                                no_repeat_ngram_size=2)

        result_top_k_sampling = happy_gen.generate_text(input_txt, args=top_k_sampling_settings)

        #전처리 필요함 GenerationResult(text=' 
        result_top_k_sampling_re = result_top_k_sampling.text
        result_top_k_sampling_re.strip("'")
        result_top_k_sampling_re.lstrip("GenerationResult(text=' ")
        result_top_k_sampling_re = re.sub(r"\n", "", result_top_k_sampling_re)


        resp.append(result_top_k_sampling_re)
    return resp

In [5]:
def ExceptionInput(input_txt):
    if (input_txt.isdigit()):
        print("An incorrect input was detected. Try again.")

    elif (len(input_txt) <= 8):
        print("The sentence is too short. Try again.")

    elif (input_txt == None):
        print("Try again.")
    else:
        pass

    return input_txt

In [6]:
def Step_01():
    print("Step #1")
    print("I am using Common App OR Coalition App")
    questions = {}
    questions['CommonApp'] = 'CommonApp'
    questions['CoalitionApp'] = 'CoalitionApp'
    
    try:
        # Let user select a script
        selected_app_input = selectFromDict(questions, 'Question')
    except:
        selected_app_input = selectFromDict(questions, 'Question')
        ExceptionInput(selected_prompt_input)
        Step_01()
    return selected_app_input

In [7]:
#Step_01()

In [8]:
def Step_02():
    print("Step #2")
    print("Select prompt")
    questions = {}
    questions['prompt01'] = 'Some students have a background, identity, interest, or talent that is so meaningful they believe their application would be incomplete without it. If this sounds like you, then please share your story.'
    questions['prompt02'] = 'The lessons we take from obstacles we encounter can be fundamental to later success. Recount a time when you faced a challenge, setback, or failure. How did it affect you, and what did you learn from the experience?'
    questions['prompt03'] = 'Reflect on a time when you questioned or challenged a belief or idea. What prompted your thinking? What was the outcome?'
    questions['prompt04'] = 'Reflect on something that someone has done for you that has made you happy or thankful in a surprising way. How has this gratitude affected or motivated you?'
    questions['prompt05'] = 'Discuss an accomplishment, event, or realization that sparked a period of personal growth and a new understanding of yourself or others.'
    questions['prompt06'] = 'Describe a topic, idea, or concept you find so engaging that it makes you lose all track of time. Why does it captivate you? What or who do you turn to when you want to learn more?'
    questions['prompt07'] = "Share an essay on any topic of your choice. It can be one you've already written, one that responds to a different prompt, or one of your own design."
    
    
    try:
        # Let user select a script
        selected_prompt_input = selectFromDict(questions, 'Question')
    except:
        selected_prompt_input = selectFromDict(questions, 'Question')
        ExceptionInput(selected_prompt_input)
        Step_02()
    return selected_prompt_input

In [9]:
#Step_02()

In [10]:
# introduction
def Step_03(Selected_topic_input):
    print("")
    print("Step #3")
    print("You choese to write about ", Selected_topic_input)
    print("How does your story begin?")
    print("Where and when?")
    print("Who are in it?")
    print("What happens in the beginning?")
    print("Your response here. Minimum 50 words.")
    
    try:
        intro_input = input()
    except:
        intro_input = input()
        ExceptionInput(intro_input)
        Step_03(Selected_topic_input)
        
        
    #intro_input = input()
    print("processing...")
    
    return intro_input

<!-- nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:36:24_Pacific_Standard_Time_2022
Cuda compilation tools, release 11.6, V11.6.124 Build cuda_11.6.r11.6/compiler.31057947_0 -->

# pytorch install for window
conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch

# install linux
pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

In [11]:
#Summarization
import torch
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
model = T5ForConditionalGeneration.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')
device = torch.device('cpu')

In [12]:
def summarization(input_txt):
    preprocess_text = input_txt.strip().replace("\n","")
    t5_prepared_Text = "summarize: "+preprocess_text
    #print ("original text preprocessed: \n", preprocess_text)

    tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt").to(device)


    # summmarize 
    summary_ids = model.generate(tokenized_text,
                                        num_beams=4,
                                        no_repeat_ngram_size=2,
                                        min_length=30,
                                        max_length=50, # Msx 2 senteces
                                        early_stopping=True)

    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output

In [13]:
# test summarization
input_txt_test = """I add the critically measured sugary tea mixture to the gallon jar containing the slimy, white, disc-shaped layers of the symbiotic culture of bacteria and yeast.
Now to wait.  After exactly seven days, I pour the liquid into a fermentation-grade glass bottle with a ratio of 20% pomegranate juice and 80% fermented tea. I place it on my kitchen counter, periodically checking it to relieve the built-up CO2.
Finally, after an additional seventy-two hours, the time comes to try it. I crack the seal on the bottle, leaning over to smell what I assume will be a tangy, fruity, delicious pomegranate solution. and it smells like rotten eggs. The insufferable stench fills my nostrils and crushes my confidence. I'm momentarily taken aback, unable to understand how I went wrong when I followed the recipe perfectly. """

summarized_txt = summarization(input_txt_test)

print(summarized_txt)

pomegranate juice and 80% fermented tea poured into a glass bottle. after seventy-two hours, the time comes to try it, and it smells like rotten eggs.


In [14]:
# Rising Action
# input_txt : 학생이 작성한 Original Text
def Step_04(input_txt):
    print("")
    print("Step #4")
    print("What happens next? AI's ideas to get you inspired!")
    gen_sentence = GenText(input_txt)
    print("")
    print(gen_sentence)
    print("")
    print("Inspired? Let's work on the rising action. What happens after the intro? -Any conflicts, problems, or questions you've dealt with?")
    print("Your response here. Minimum 50 words.")
    
    try:
        user_response = input()
    except:
        user_response = input()
        ExceptionInput(user_response)
        Step_04(input_txt)
    
    # user_response = input()
    
    #gen_sentence : 생성된 rising aciton  6개의 문장들
    #user_response : 추가입력하는 사용자  rising action 입력문장
    return gen_sentence, user_response

In [15]:
# intro_ori_input_sent : intro 입력 원본문장
# intro_input_summarised_txt: 이미 요약된 문장으로 intro 부분
# risingAction_input_txt: Rising action 입력문장
def Step_05(intro_ori_input_sent, intro_input_summarised_txt, risingAction_input_txt):
    print("")
    print("Step #5")
    print("Your story so far.")
    risingAciton_summary_re = summarization(risingAction_input_txt)
    # Intro + Rising Action 내용 3-4문장 간추림
    print(intro_input_summarised_txt + risingAciton_summary_re)
    pre_sum_climax_sentences = intro_input_summarised_txt + risingAciton_summary_re
    
    print("")
    #Generate 되는 문장들은 학생의 original writing들 기반 (summary 아니고)
    gen_climax_sentence = GenText(intro_ori_input_sent + risingAction_input_txt)
    print(gen_climax_sentence)
    
    print("What happens next? AI's ideas to get you inspired!")
    print("")
    print("Describe the most dramatic moment in the story. What happens at the story's peak? How did your conflict, problem, or question resolve?")
    print("Your response here. Minimum 50 words.")
    
    try:
        climax_input = input()
    except:
        climax_input = input()
        ExceptionInput(climax_input)
        Step_05(intro_ori_input_sent, intro_input_summarised_txt, risingAction_input_txt)
    
    # climax_input = input()
    print("Your Climax input:", climax_input)
    
    summarised_climax = summarization(climax_input)
    
    # risingAciton_summary_re : summary rising action
    # gen_climax_sentence : 생성된 Climax idea sentences
    # climax_input : Use input climax
    # summarised_climax : summary climax
    return risingAciton_summary_re, gen_climax_sentence, climax_input, summarised_climax
    

In [16]:
def Step_06(sum_intro, sum_rasingAciton, sum_climax, intro_ori_input_sent, risingAction_input_txt, climax_input_txt):
    print("")
    print("Step #6")
    print("How did your conflict, problem, or question resolve?")
    # show summary of Intro + Rasing Action + Climax
    step_6_que_re = sum_intro + sum_rasingAciton + sum_climax
    #print(step_6_que_re)
    print("")
    print("What happens next? AI's ideas to get you inspired!")
    gen_resolution = GenText(intro_ori_input_sent + risingAction_input_txt + climax_input_txt)
    print("Generated resolution sentences :", gen_resolution)
    print("")
    print("""Describe how your story ends. What happens at the end of your story? -How did this experience change your perspective?""")
    print("Your response here. Minimum 50 words.")
    
    try:
        Resolution_input = input()
    except:
        Resolution_input = input()
        ExceptionInput(Resolution_input)
        Step_06(sum_intro, sum_rasingAciton, sum_climax, intro_ori_input_sent, risingAction_input_txt, climax_input_txt)
    
    # Resolution_input = input()
    print("Your resolution input :", Resolution_input)
    
    # gen_resolution : 생성한 Resolution sentences
    # Resolution_input : User input resolution
    return gen_resolution, Resolution_input

In [1]:
doc = """
         Supervised learning is the machine learning task of 
         learning a function that maps an input to an output based 
         on example input-output pairs.[1] It infers a function 
         from labeled training data consisting of a set of 
         training examples.[2] In supervised learning, each 
         example is a pair consisting of an input object 
         (typically a vector) and a desired output value (also 
         called the supervisory signal). A supervised learning 
         algorithm analyzes the training data and produces an 
         inferred function, which can be used for mapping new 
         examples. An optimal scenario will allow for the algorithm 
         to correctly determine the class labels for unseen 
         instances. This requires the learning algorithm to  
         generalize from the training data to unseen situations 
         in a 'reasonable' way (see inductive bias).
      """

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

n_gram_range = (1, 1)
stop_words = "english"

# Extract candidate words/phrases
count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([doc])
candidates = count.get_feature_names()

C:\Users\cacki\Anaconda3\envs\py37NLP\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# https://www.sbert.net/docs/pretrained_models.html#average-word-embeddings-models

from sentence_transformers import SentenceTransformer

model = SentenceTransformer('xlm-r-distilroberta-base-paraphase-v1')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [1]:

from sklearn.metrics.pairwise import cosine_similarity

top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]

NameError: name 'doc_embedding' is not defined

In [ ]:
def KeyTopic():
    

In [17]:
def Run():
    Step_01()
    
    print("")
    
    selected_topic = Step_02()
    
    print("")
    
    #intro_input_txt:  학생이 작성한 intro sentence
    intro_input_txt = Step_03(selected_topic)
    
    print("")
    
    #intro summary sentence
    intro_summarized_txt = summarization(intro_input_txt)
    
    print("")
    
    print('summary intro :', intro_summarized_txt)
    
    print("")

    result = Step_04(intro_input_txt)
    #gen_sentence  > result[0]: 생성된 6개의 risng action 문장들
    #user_response  > result[1]: 사용자 rising action입력문장
    
    print("")
    
    step_5_re = Step_05(intro_input_txt, intro_summarized_txt, result[1])
    print("Generated 6 sentences of the climax idea", step_5_re[1]) # 생성된 Climax idea sentences
    print("")
    print("User climax input sentence :", step_5_re[2]) # User climax input
    print("")
    
    sum_intro = intro_summarized_txt
    sum_rasingAciton = step_5_re[0]
    sum_climax = step_5_re[2] 
    intro_ori_input_sent = intro_input_txt
    risingAction_input_txt = result[1]
    climax_input_txt = step_5_re[2]
    
    gen_resolution_re = Step_06(sum_intro, sum_rasingAciton, sum_climax, intro_ori_input_sent, risingAction_input_txt, climax_input_txt)
    print("")
    # gen_resolution_re[0] : 생성한 Resolution sentences
    # gen_resolution_re[1] : User input resolution
    print("Generated the resolution sentences :", gen_resolution_re[0])
    print("")
    print("User input resolution sentence :", gen_resolution_re[1])
    print("")
    
    return gen_resolution_re[0], gen_resolution_re[1]

In [18]:
Run()


Step #1
I am using Common App OR Coalition App
Select a Question:
1) CommonApp
2) CoalitionApp
Question: 1
Selected Question: CommonApp

Step #2
Select prompt
Select a Question:
1) prompt01
2) prompt02
3) prompt03
4) prompt04
5) prompt05
6) prompt06
7) prompt07
Question: 2
Selected Question: The lessons we take from obstacles we encounter can be fundamental to later success. Recount a time when you faced a challenge, setback, or failure. How did it affect you, and what did you learn from the experience?


Step #3
You choese to write about  The lessons we take from obstacles we encounter can be fundamental to later success. Recount a time when you faced a challenge, setback, or failure. How did it affect you, and what did you learn from the experience?
How does your story begin?
Where and when?
Who are in it?
What happens in the beginning?
Your response here. Minimum 50 words.
I add the critically measured sugary tea mixture to the gallon jar containing the slimy, white, disc-shaped lay

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



summary intro : the sugary tea mixture contains the slimy, white, disc-shaped layers of the symbiotic culture of bacteria and yeast.


Step #4
What happens next? AI's ideas to get you inspired!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



['I fill my funnel with the sugared tea and pour it slowly into the jar. I hold the funnel in my mouth while I drink the tea. It is like drinking an alcoholic drink. The tea is sweet and the sugar is in the form of a syrup. But the taste is not sweet. That is the real taste. When I am done, I put the empty funnel back into my jar and put it into a plastic bag. Then I carefully, carefully clean off the lid', ' I use a funnel to pour the mixture into the jar. Now I have a wonderful mixture of sugared tea and the culture.I pour about 2 inches of this mixture onto the surface of my fermenting sourdough starter and leave it bare, waiting for it to ferment. This is an important step so that the yeast can use the added sugar from the tea to produce alcohol (which is what fermentation is all about).Instead of pouring the suger mixture directly into my sour', "The jar is sealed tightly, and I return to my office, to a mug of steaming tea, with a bowl of fresh fruit to sit on the desk blotter. 

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated 6 sentences of the climax idea [" The smell is so bad that I can't even imagine what it will do to my clothes when it dries.So, I surrender to a bitter disappointment. Instead of a delicious, tanginess-filled pom-granated tea, what does I get is a rancid, sour, rotten, musty concoction that takes me hours to clean up.This is the real purpose of my experiment.I'm not trying to make tea.Instead, my goal is to prove that", ' "I knew you\'d want to drink from the original bottle." It\'s my dear friend, my mentor, and one of my earliest critics, Dr. Scott Leifheit. He\'s also a little miffed that I have yet to accept his invitation to attend his annual Christmas party this year. "I don\'t need your money, David. You know I don’t drink the stuff." I can\'t vouch for his sobriety, but I\'d be surprised', ' In this moment, I feel shame. Shame for having underestimated my assistant\'s ability to follow instructions. For being so presumptuous and arrogant to think I could do any better

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated resolution sentences : ["The most important thing I've learned from my SCOBY CUP journey is to carefully consider the person I am and what they like. For many people, SCBAs are a part of their family history. They are part and parcel of how they grew up, how their parents grew their food, or how the food their families grew was prepared. So, when people are presented with being forced to make a SCABY, they tend to have a hard time accepting it as the", 'Dr. Michael Jacobson, an emergency room physician and clinical instructor at the University of North Carolina School of Medicine, was one of my early SCABF fellow students. He recalls:"I started learning about SCBs about the same time that I learned about kimchi, which was in 1991, about eight years before I started working in emergency medicine. When I saw the importance of probiotics, as well as the growing body of evidence that SCB and kuksu', 'I\'ve seen a lot of versions of this "fruity" kocha, but this one is unique in t

(["The most important thing I've learned from my SCOBY CUP journey is to carefully consider the person I am and what they like. For many people, SCBAs are a part of their family history. They are part and parcel of how they grew up, how their parents grew their food, or how the food their families grew was prepared. So, when people are presented with being forced to make a SCABY, they tend to have a hard time accepting it as the",
  'Dr. Michael Jacobson, an emergency room physician and clinical instructor at the University of North Carolina School of Medicine, was one of my early SCABF fellow students. He recalls:"I started learning about SCBs about the same time that I learned about kimchi, which was in 1991, about eight years before I started working in emergency medicine. When I saw the importance of probiotics, as well as the growing body of evidence that SCB and kuksu',
  'I\'ve seen a lot of versions of this "fruity" kocha, but this one is unique in that it also tastes like pure